https://data.seattle.gov/Public-Safety/Terry-Stops/28ny-9ts8

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import os, sys

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
! ls

README.md
analysis
data
images
report


In [8]:
pd.read_csv('a')

FileNotFoundError: [Errno 2] No such file or directory: 'a'

In [16]:
pd.read_csv('terry-stops.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'terry-stops.csv'

In [14]:
#path_to_raw = os.path.join()
df = pd.read_csv(os.path.join('..', 'data', 'raw', 'terry-stops'))

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\raw\\terry-stops'